# DQNの解説

![image.png](attachment:image.png)

画像引用:  
https://qiita.com/sugulu/items/3c7d6cbe600d455e853b

### DQNの特徴
- Q学習において状態行動テーブルを関数で表したもの.
- 離散的な行動を扱うことができる.

参考:  
http://blog.syundo.org/post/20171208-reinforcement-learning-dqn-and-impl/

### OpenAI gymのインストール

githubのレポジトリを参考に, gymモジュールをインストールしてください.  
https://github.com/openai/gym

In [1]:
import gym
import numpy as np
import renom as rm
import matplotlib.pyplot as plt
from renom.utility.initializer import Gaussian
from renom.cuda import set_cuda_active
from renom_rl.discrete.dqn import DQN
from renom_rl.env import BaseEnv
from gym.core import Env
from PIL import Image
from logging import getLogger, StreamHandler, DEBUG, FileHandler

set_cuda_active(True)
env = gym.make('BreakoutNoFrameskip-v4')

class CustomEnv(BaseEnv):
    
    def __init__(self, env):
        self.env = env
        self.action_shape = 4
        self.state_shape = (4, 84, 84)
        self.previous_frames = []
        self._reset_flag = True
        self._last_live = 5
        super(CustomEnv, self).__init__()
    
    def reset(self):
        if self._reset_flag:
            self._reset_flag = False
            self.env.reset()
        n_step = np.random.randint(4, 32+1)
        for _ in range(n_step):
            state, _, _ = self.step(self.env.action_space.sample())
        return state
    
    def sample(self):
        return self.env.action_space.sample()
    
    def render(self):
        self.env.render()

    def _preprocess(self, state):
        resized_image = Image.fromarray(state).resize((84, 110)).convert('L')
        image_array = np.asarray(resized_image)/255.
        final_image = image_array[26:110]
        # Confirm that the image is processed correctly.
        # Image.fromarray(np.clip(final_image.reshape(84, 84)*255, 0, 255).astype(np.uint8)).save("test.png")
        return final_image
    
    def step(self, action):
        state_list = []
        reward_list = []
        terminal = False
        for _ in range(4):
            # Use last frame. Other frames will be skipped.
            s, r, t, info = self.env.step(action)
            state = self._preprocess(s)
            reward_list.append(r)
            if self._last_live > info["ale.lives"]:
                t = True
                self._last_live = info["ale.lives"]
                if self._last_live > 0:
                    self._reset_flag = False
                else:
                    self._last_live = 5
                    self._reset_flag = True
            if t:
                terminal = True
                
        if len(self.previous_frames) > 3:
            self.previous_frames = self.previous_frames[1:] + [state]
        else:
            self.previous_frames += [state]
        state = np.stack(self.previous_frames)
        return state, np.array(np.sum(reward_list) > 0), terminal
    
custom_env = CustomEnv(env)
q_network = rm.Sequential([rm.Conv2d(32, filter=8, stride=4, ignore_bias=True),
                           rm.Relu(),
                           rm.Conv2d(64, filter=4, stride=2, ignore_bias=True),
                           rm.Relu(),
                           rm.Conv2d(64, filter=3, stride=1, ignore_bias=True),
                           rm.Relu(), 
                           rm.Flatten(), 
                           rm.Dense(512, ignore_bias=True),
                           rm.Relu(),
                           rm.Dense(custom_env.action_shape, ignore_bias=True)])

In [2]:
model = DQN(custom_env, q_network)

In [3]:
result = model.fit(render=False, greedy_step=1000000, random_step=5000, update_period=10000)

Run random 5000 step for storing experiences


epoch 001 avg_loss: 0.005 total_reward [train:20.000 test:11.000] e-greedy:0.002: 100%|██████████| 2000/2000 [00:30<00:00, 64.95it/s]
epoch 002 avg_loss: 0.004 total_reward [train:21.000 test:16.000] e-greedy:0.004: 100%|██████████| 2000/2000 [00:31<00:00, 64.37it/s]
epoch 003 avg_loss: 0.003 total_reward [train:15.000 test:13.000] e-greedy:0.005: 100%|██████████| 2000/2000 [00:31<00:00, 63.59it/s]
epoch 004 avg_loss: 0.003 total_reward [train:24.000 test:16.000] e-greedy:0.007: 100%|██████████| 2000/2000 [00:31<00:00, 64.08it/s]
epoch 005 avg_loss: 0.002 total_reward [train:18.000 test:9.000] e-greedy:0.009: 100%|██████████| 2000/2000 [00:31<00:00, 63.96it/s]
epoch 006 each step reward:0.000:   1%|          | 14/2000 [00:00<00:29, 68.12it/s]

Update 16.0


epoch 006 avg_loss: 0.004 total_reward [train:23.000 test:1.000] e-greedy:0.011: 100%|██████████| 2000/2000 [00:31<00:00, 63.55it/s]
epoch 007 avg_loss: 0.003 total_reward [train:22.000 test:14.000] e-greedy:0.013: 100%|██████████| 2000/2000 [00:30<00:00, 65.21it/s]
epoch 008 avg_loss: 0.003 total_reward [train:26.000 test:15.000] e-greedy:0.014: 100%|██████████| 2000/2000 [00:30<00:00, 64.89it/s]
epoch 009 avg_loss: 0.003 total_reward [train:17.000 test:10.000] e-greedy:0.016: 100%|██████████| 2000/2000 [00:31<00:00, 69.96it/s]
epoch 010 avg_loss: 0.003 total_reward [train:30.000 test:17.000] e-greedy:0.018: 100%|██████████| 2000/2000 [00:30<00:00, 65.21it/s]
epoch 011 each step reward:0.000:   1%|          | 12/2000 [00:00<00:26, 73.91it/s]

Update 17.0


...

epoch 311 avg_loss: 0.040 total_reward [train:38.000 test:22.000] e-greedy:0.560: 100%|██████████| 2000/2000 [00:29<00:00, 66.70it/s]
epoch 312 avg_loss: 0.039 total_reward [train:41.000 test:20.000] e-greedy:0.562: 100%|██████████| 2000/2000 [00:30<00:00, 66.51it/s]
epoch 313 avg_loss: 0.037 total_reward [train:41.000 test:24.000] e-greedy:0.563: 100%|██████████| 2000/2000 [00:29<00:00, 67.77it/s]
epoch 314 avg_loss: 0.038 total_reward [train:42.000 test:23.000] e-greedy:0.565: 100%|██████████| 2000/2000 [00:30<00:00, 66.36it/s]
epoch 315 avg_loss: 0.038 total_reward [train:41.000 test:24.000] e-greedy:0.567: 100%|██████████| 2000/2000 [00:31<00:00, 71.81it/s]
epoch 316 each step reward:0.000:   1%|          | 14/2000 [00:00<00:27, 71.96it/s]

Update 24.0


epoch 316 avg_loss: 0.044 total_reward [train:39.000 test:23.000] e-greedy:0.569: 100%|██████████| 2000/2000 [00:31<00:00, 63.97it/s]
epoch 317 avg_loss: 0.044 total_reward [train:45.000 test:23.000] e-greedy:0.571: 100%|██████████| 2000/2000 [00:31<00:00, 63.96it/s]
epoch 318 each step reward:37.000:  90%|█████████ | 1800/2000 [00:30<00:03, 55.75it/s]

KeyboardInterrupt: 

epoch 318 each step reward:37.000:  90%|█████████ | 1801/2000 [00:50<00:03, 55.75it/s]

In [ ]:
q_network.save("dqn_exp5.h5")
# model = DQN(custom_env, q_network)

In [ ]:
model.test(render=True)

In [ ]:
import time
start_t = time.time()
a = np.random.permutation(int(1e1))
print(time.time()-start_t)


# 